# Reproduce selected figures from Hills et al. (2022)
## Radar attenuation demonstrates advective cooling in the Siple Coast ice streams

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

# If the icetemperature library is not in your PYTHONPATH, you will not be able to load those functions
# Check and update here if necessary
import sys
cs_dir = '../'
if cs_dir not in sys.path:
    sys.path.append(cs_dir)

# Import the ice temperature model and relevant constants
from iceotherm.lib.numerical_model import ice_temperature
from iceotherm.lib.analytical_solutions import Robin_T, Meyer_T, Perol_T
from iceotherm.lib.ice_properties import conductivity, heat_capacity, rate_factor
from iceotherm.lib.constants import constants
const = constants()

In [ ]:
# Create a function wrapper around the model for this specific use case
def numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                    dS=0.0,p=12.3,
                    plane_strain=False,ϵ_xy=0.,A_xy='full',
                    dTs=0.,dH=0.,da=0.,
                    tol=1e-8,dt=1.):
    """
    Parameters
    ---
    adot: accumulation rate
    Ts:   surface temperature
    H:    ice thickness
    qgeo: geothermal flux
    Udef: deformational velocity
    Uslide: sliding velocity
    dS:   surface slope
    p:    shape factor for vertical flow
    plane_strain: heat source in the shear margin
    ϵ_xy: plane strain rate
    A_xy: plane strain rate factor
    dTs:  gradient in surface temperature
    dH:   thickness gradient
    da:   accumulation gradient
    tol:  numerical tolerance
    dt:   time step (years)
    
    Output
    ---
    m:    model class
    """

    # Initialize the model class
    m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo,
                        p=p,dTs=dTs,dH=dH,da=da,dS=dS,
                        A_xy=A_xy,eps_xy=ϵ_xy)
    # Velocity terms
    m.Udef = Udef/const.spy
    m.Uslide = Uslide/const.spy

    # Set the process inclusion flags
    if plane_strain:
        m.flags.append('plane_strain')
    m.flags.append('long_advection')
    m.flags.append('vertical_shear')    
    m.flags.append('temp-dependent')
    
    # Get the initial conditions and ice properties
    m.initial_conditions()
    m.k = conductivity(m.T.copy(),m.rho)
    m.Cp = heat_capacity(m.T.copy())
    
    # Setup source terms and numerical stencil
    m.source_terms()
    m.stencil(dt*const.spy)
    m.tol=tol
    
    # Run to steady state
    m.numerical_to_steady_state()
    
    return m

In [ ]:
### Dragon Shear Margin ###
# -----------------------------------------------------------------------

# Set location parameters
Ts = -26.0
H = 1050.
adot = .0698
qgeo = 0.0633
Udef = 0.
Uslide = 180.
eps_xy = 4.75e-2
dS = 1.3e-3*180./np.pi
print('Dragon; H:',H,'T_surface:',Ts,'Accumulation Rate:',round(adot,3))

# Initialize the model
m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo,eps_xy=eps_xy)
# Robin analytical solution
TRdrag,Mrate = Robin_T(m)
# Meyer analytical solution
TM_soft = Meyer_T(m,T_bulk=None)
TM_soft += (max(m.z)-m.z)*const.rho*const.g*const.beta
Tm = const.beta*const.rho*const.g*m.H
TM_stiff = Meyer_T(m)
TM_stiff += (max(m.z)-m.z)*const.rho*const.g*const.beta
# Run the model for the soft case
m_drag = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                        dTs=12.1e-6,dH=-0.95e-3,da=-0.31e-6,
                        ϵ_xy=eps_xy,tol=1e-9,plane_strain=True,dS=dS,
                        A_xy=rate_factor(0.))
# Run the model for the stiff case
m_drag_visc = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                        dTs=12.1e-6,dH=-0.95e-3,da=-0.31e-6,
                        ϵ_xy=eps_xy,tol=1e-9,plane_strain=True,dS=dS)


### Kamb Ice Stream ###
# -----------------------------------------------------------------------

# Set location parameters
Ts = -25.5
H = 949.
adot = .0858
qgeo = .0663
Uslide = 300.
print('Kamb Ice Stream; H:',H,'T_surface:',Ts,'Accumulation Rate:',round(adot,3))

# Initialize the model
m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo)
# Robin analytical solution
TRkamb,Mrate = Robin_T(m)
# Set a percentile for 'best fit' of the longitudinal advection heat sink (Hills et al., 2022)
Λ_frac = 0.735
# Run the model for the best-fit case
m_kis = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                        dTs=13.4e-6*Λ_frac,dH=-3.45e-3*Λ_frac,da=-0.27e-6*Λ_frac,dS=.15)
# Run the model in a transient case for Kamb Ice Stream stagnation
ntsteps = 100
m_kis.adot_s = np.array([m_kis.adot]*ntsteps)
m_kis.Ts_s = np.array([m_kis.Ts]*ntsteps)
dt=1.
m_kis.ts = np.arange(0,dt*ntsteps,dt)*const.spy
m_kis.Udef = 0./const.spy
m_kis.Uslide = 10./const.spy
m_kis.numerical_transient()
    
### Bindschadler Ice Stream ###
# -----------------------------------------------------------------------

# Set location parameters
Ts = -24.4
H = 1086.
adot = 0.111
qgeo = .0604
Udef = 0.
Uslide = 375.
print('Bindschadler Ice Stream; H:',H,'T_surface:',Ts,'Accumulation Rate:',round(adot,3))

# Initialize the model
m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo)
# Robin analytical solution
TRbind,Mrate = Robin_T(m)
# Set a percentile for 'best fit' of the longitudinal advection heat sink (Hills et al., 2022)
Λ_frac = 0.596
# Run the model for the best-fit case
m_bis = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                        dTs=10.1e-6*Λ_frac,dH=-3.12e-3*Λ_frac,da=-0.15e-6*Λ_frac,dS=.15)
    
### Siple Dome ###
# -----------------------------------------------------------------------

# Set location parameters
Ts = -24.6
H = 1005.
adot = 0.121
qgeo = .0653
print('Siple Dome; H:',H,'T_surface:',Ts,'Accumulation Rate:',round(adot,3))

# Initialize the model
m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo)
# Robin analytical solution
TRsiple,Mrate = Robin_T(m)
# Run the model
m_siple = numerical_model(adot,Ts,H,qgeo,0.,0.)

In [ ]:
fig = plt.figure(figsize=(6,6))

### Bindschadler Ice Stream ###
# -----------------------------------------------------------------------

ax1 = plt.subplot(221)
ax1.tick_params(labelbottom=False)
plt.ylim(0,1150)
plt.xlim(-30,2)
ax1.set_title('Bindschadler Ice Stream')
l1, = ax1.plot(TRbind,m_bis.z,'k',zorder=2)  
ax1.plot(m_bis.T,m_bis.z,'steelblue')

### Kamb Ice Stream ###
# -----------------------------------------------------------------------

ax2 = plt.subplot(222)
ax2.tick_params(labelleft=False,labelbottom=False)
plt.ylim(0,1150)
plt.xlim(-30,2)
ax2.set_title('Kamb Ice Stream')
l1, = ax2.plot(TRkamb,m_kis.z,'k',zorder=2)  
l4, = ax2.plot(m_kis.T_steady,m_kis.z,'steelblue')
l5, = ax2.plot(m_kis.T,m_kis.z,'navy')

### Siple Dome ###
# -----------------------------------------------------------------------

ax3 = plt.subplot(223)
plt.ylabel('Height Above Bed (m)')
plt.xlabel('Temperature ($^\circ$C)')
plt.ylim(0,1150)
plt.xlim(-30,2)
ax3.set_title('Siple Dome')
ax3.plot(TRsiple,m_siple.z,'k',zorder=1)
ax3.plot(m_siple.T,m_siple.z,'steelblue')

### Dragon Shear Margin ###
# -----------------------------------------------------------------------

ax4 = plt.subplot(224)
ax4.tick_params(labelleft=False)
plt.xlabel('Temperature ($^\circ$C)')
plt.ylim(0,1150)
plt.xlim(-30,2)
ax4.set_title('Dragon Shear Margin')
ax4.plot(TRdrag,m_drag.z,'k',zorder=1)
ax4.fill_betweenx(m_drag.z,TM_soft,TM_stiff,color='maroon',zorder=2,alpha=0.1)
l2, = ax4.plot(TM_soft,m_drag.z,'maroon',zorder=2)
ax4.plot(TM_stiff,m_drag.z,'maroon',zorder=2)
ax4.plot(TRdrag,m_drag.z,'k',zorder=1)
ax4.fill_betweenx(m_drag.z,TM_soft,TM_stiff,color='maroon',zorder=2,alpha=0.1)
l2, = ax4.plot(TM_soft,m_drag.z,'maroon',zorder=2)
ax4.plot(TM_stiff,m_drag.z,'maroon',zorder=2)
ax4.fill_betweenx(m_drag.z,m_drag.T,m_drag_visc.T,color='steelblue',alpha=0.1)
ax4.plot(m_drag.T,m_drag.z,'steelblue')
ax4.plot(m_drag_visc.T,m_drag.z,'steelblue')

ax3.legend([l1,l2,l4,l5],
           ['Robin (1955)','Meyer (2018)','Weertman (1968)\nBest Fit','Kamb Stagnation'],
          fontsize=8)

plt.tight_layout()